# Data import

In [1]:
## Import packages we will use 
import json
import pprint 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import collections 
import tensorflow as tf
from sklearn.manifold import TSNE

import json
import yaml  # https://pypi.python.org/pypi/PyYAML
from boltons.iterutils import remap  # https://pypi.python.org/pypi/boltons

#http://sedimental.org/remap.html#

In [2]:
## Read the data, one line (= one json) at a time 
data = []
with open('products-sample.json','r') as f:
    for line in f:
        data.append(json.loads(line))

In [3]:
#keep just the correct shoe variant in the record
for line in data:
    variant_id = int(line['variant_id'].replace(line['product_id'],""))-1
    line['variant_data'] = line['variant_data'][variant_id]

In [4]:
def visit(path, key, value):
    if key == 'date':
        return key, pd.to_datetime(value)
    return key, value
data = remap(data, visit=visit)

In [5]:
from collections import Iterable
def flatten(coll):
    l=[]
    for i in coll:
            if not isinstance(i, basestring):
                for subc in flatten(i):
                    yield subc
            else:
                yield i

In [6]:
def visit(path, key, value):
    if key == 'params':
        li=[]
        for param in value:
            l = list(param['values'].values())
            if not isinstance(l[0], str):
                l = [item for sublist in l for item in sublist]
            l = li.append((param['id'],l[0]))
        return key, dict(li)
    return key, value
data = remap(data, visit=visit)

In [7]:
data[0]

{'brand': {'brand_id': 'NIKE', 'title': {'C': 'Nike'}},
 'brief_plain': {'C': '\nNike Reax 8 TR je pánská fitness obuv. Tenisky navržené ke sportu perfektně tlumí dopady. Maximálně prodyšný materiál zajišťuje rychlejší odvod vlhkosti z boty ven.\n'},
 'date': Timestamp('2017-11-15 04:57:14.392000'),
 'product_id': '1070393',
 'product_type': {'id': 'NM006', 'title': {'C': 'Obuv'}},
 'title_full': {'C': 'Nike Reax 8 TR'},
 'user_id': '485d191a800313d36115340fa586aa16',
 'variant_data': {'availability': {'CZ': {'clearance_outlets': [0, 0],
    'external_availability': 0,
    'external_term_days': 999,
    'is_available': True,
    'numeric': 5,
    'our_availability': 1,
    'outlets': {'1': 0,
     '2': 0,
     '3': 0,
     '4': 0,
     '5': 0,
     '6': 0,
     '7': 0,
     '8': 0,
     '9': 0},
    'status': 'A1',
    'supplier_availability': 0,
    'supplier_term_days': 999}},
  'labels': {'CZ10MA': [{'label_id': 'BFTE'}, {'label_id': 'NEW'}]},
  'params': {'AUTUMN_SHOES': 'YES',
   

Co to je promotion amount nevim je to neco jako sleva, je to u kazdyho, takze to muzeme brat jako indikator, ze je neco zlevneny

jsou nektery params pro kazdou variantu stejny (treba type of shoes?) budeme muset prozkoumat, ktery veci jsou stejny napric vsema variantama.

Nevim zatim jak presne funguje all year/season field, takze to tam zatim neni.. asi to bude kombinace toho jestli jsou summer nebo ne a toho co je tam o tom season type..

Podobne nevim jak poznat pro koho jsou ty boty obecne, key determination, kde se prave pise mens nebo kids nekde chybi, takze to bude taky nejaky slozitejsi.. muzu to vykoukavat z dat, ale pokud nam to reknou bude to samozrejme rychlejsi..

Pak se koukam na to ktery keys v elems se vyskytujou vsude. Jsou to jen tri:

In [8]:
l=[]
for line in data:
    a=set(line['variant_data']['params'].keys())
    l.append(a)
set.intersection(*l)

{'COLOR', 'SIZE_EUR', 'TYPE_OF_SHOES'}

In [9]:
numeric_vector = []
for line in data:
    numeric_vector.append({
        'user_id': line['user_id'],
        'product_id': line['product_id'],
        'variant_id': line['variant_id'],
        'date': line['date'],
        'brand': line['brand']['brand_id'],
        'price': int(line['variant_data']['price']['CZ1000']['price']),
        'promotion_amount': int(line['variant_data']['price']['CZ1000']['promotion_amount']),
        'Shoe_size': line['variant_data']['params']['SIZE_EUR'],
        'Type_of_shoes': line['variant_data']['params']['TYPE_OF_SHOES'],
        'Color': line['variant_data']['params']['COLOR']
    })
df = pd.DataFrame(numeric_vector).sort_values('date').reset_index().drop("index",axis=1)

In [10]:
df

,Color,Shoe_size,Type_of_shoes,brand,date,price,product_id,promotion_amount,user_id,variant_id
0,BLACK,45.0,ANKLE BOOTS,MERRELL,2017-11-15 00:21:07.882,2759,1067675,47,f221e7fb514f76d136cae7e121d21d30,1067675002
1,WHITE,37.0,ANKLE BOOTS,TAMARIS,2017-11-15 01:37:27.912,2699,1057170,0,f71371e961eb4bd9ecafa0d716b81195,1057170002
2,BLACK/GREEN,41.0,RUNNING SHOES,MIZUNO,2017-11-15 02:26:44.605,2059,1009824,99,9bb5edc8ddfd494cb86c77613eac996c,1009824001
3,GREEN,40.0,TREKING SHOES,MERRELL,2017-11-15 02:59:22.076,2579,1016868,111,d34eeecc402a19b2e3fb606e692bb8f1,1016868006
4,TURQUOISE,37.0,GUM BOOT,MEI,2017-11-15 03:47:28.749,1099,847829,56,eedd8640f7f9ba5f509ccbd22cffe5e2,847829008
5,ORANGE,39.0,GUM BOOT,MEI,2017-11-15 03:49:30.200,1099,847827,39,eedd8640f7f9ba5f509ccbd22cffe5e2,847827016
6,WHITE/BLUE,34.0,HALL SHOES,MIZUNO,2017-11-15 03:54:53.799,1689,1081678,64,4c7111b376ca2b33f592999cbc1adf88,1081678001
7,WHITE/BLACK,41.0,SPORT SHOES,NIKE,2017-11-15 03:59:44.834,3039,1072588,89,a6a488895d0e7e30a7f89ab9b5cc8feb,1072588001
8,BLACK,30.0,GUM BOOT,KAMIK,2017-11-15 04:25:04.117,879,1044140,0,25fc4b7750477c8686a127d57ca719ac,1044140001
9,PINK,36.0,SANDALS,REGATTA,2017-11-15 04:30:12.564,1159,879091,88,4a1c331bc32554a1855572ae73ed9956,879091009
